# SUDOKU SOLVER

First, we need to create the grid we want to solve.
We have to type it manually, since we don't have any user interface. 

We are using a list including 9 sublists. Each of this sublist is made of 9 integers.   
The first sublist is the first row of the sudoku grid.  
The second sublist is the second row of the sudoku grid.  
Etc...  
Up to the ninth sublist.  

We fill every empty cell with a zero.

We end up with 81 integers.

Here is an example :

In [ ]:
Grid = [
    [0, 3, 2, 0, 0, 0, 5, 0, 0],
    [4, 0, 0, 2, 0, 0, 0, 8, 0],
    [0, 0, 0, 0, 0, 0, 4, 7, 0],
    [5, 2, 0, 0, 7, 0, 0, 0, 4],
    [0, 0, 0, 1, 0, 6, 0, 0, 0],
    [7, 0, 0, 0, 5, 0, 0, 3, 9],
    [0, 9, 7, 0, 0 ,0, 0, 0, 0],
    [0, 6, 0, 0, 0, 3, 0, 0, 5],
    [0, 0, 5, 0, 0, 0, 7, 9, 0]
    ]

If we want to display the list using the `print` function, it will not do it as a grid but as a list:

In [ ]:
print(Grid)

To display it properly, we can use :

In [ ]:
print(*Grid, sep="\n")

Note: We will now use "grid" to refer to the list and "rows" to refer to the sublists.

We will work on our grid, so we need to make a copy of it, to retain the original values.
We are using a loop with the `for` function to copy one by one each row of the grid as independent copy:

In [ ]:
GridCopy = [R[:] for R in Grid]

The grid includes 81 entries from 0 to 80.  We will use the variable `N` to refer to this number.  
`N`is located in between `Grid[0]|0]` (which is the location of `N` when equals to 0) and `Grid[8][8]`(which is the location of `N` when equals to 80). 
We need to be able to identify the indexes (for the row and column) for any given `N` value.  

The first row is numbered from 0 to 8. Its index is `Grid[0]|0:8]`. `N // 9` is always equal to 0 in this row.    
The second row is numbered from 9 to 17.  Its index is `Grid[1]|0:8]`. `N // 9` is always equal to 1 in this row.   
Etc...  
Up to the ninth row which is numbered from 71 to 80.  Its index is `Grid[8]|0:8]`. `N // 9` is always equal to 8 in this row.   

As we can see, `N // 9` is always equal to the index of the row. The modulo `N % 9` is always equal to the index positioning `N` among the row (corresponding to its position as a column).

We are creating the function `IR` and `IC`:

In [ ]:
#Index (Row):
IR = lambda N : N // 9
#Index (Column):
IC = lambda N : N % 9

Note: We used`lambda` to create a simple function.  
It could also be written that way with `def`:

In [ ]:
def IR(N):
    IR = N // 9
    return IR

def IC(N):
    IC = N % 9
    return IC

We can deduce that `Grid[IR(N)][IC(N)]` determines the value of an entry at the location of `N`.  

We create another function for that:

In [ ]:
NValue = lambda G, N : G[IR(N)][IC(N)]

A sudoku grid is made of 9 subgrids, 9 rows and 9 columns. 
We need to create a function to isolate each one of the element according to `N` in order to know if `NValue(N)`is already among it or not.

The starting point of a subgrid is located at `[IR(N)] // 3 * 3` and `[IC(N)] // 3 * 3` which can be equal to 0, 3, or 6, the maximum index of a row or column being 8.
From that location, we need to select the 3 next entries in the range `[IC(N):IC(N)+3]`, then repeat the operation 2 times in the rows below.  
We create a function to return the 9 entries from the subgrid as a list:

In [ ]:
def SubGrid(G, N):
    ISR = IR(N) // 3 * 3
    ISC = IC(N) // 3 * 3
    SubGrid = []
    for R in range(0, 3):
        SubGrid += G[ISR+R][ISC:ISC+3]
    return SubGrid

We also create functions to isolate the row and column concerned by `N` as lists of 9 entries:

In [ ]:
Row = lambda G, N : G[IR(N)][0:9]
Column = lambda G, N : [R[IC(N)] for R in G]

We set different variables for the loop and set the initial value of `N` to 0.  
`Next` and `Previous` are used to determined if we are going through the grid forward or backward.

In [ ]:
Next = True
Previous = False
N = 0

The following cell contains a commented explanation of the loop we are using to solve the sudoku:

In [ ]:
while N < 81:
#The grid includes 81 entries from 0 to 80. As long as N is less than 81, the loop will be running forward or backward.

#Forwards conditions :

    if NValue(GridCopy, N) != 0 and Next:
    #We are checking if the value of the entry in the copy of the grid is 0. We use the copy because it will keep the original values and not be modified.
        N += 1
        #If N is not 0, the value is a pre-established one, and we skip it. 

    elif NValue(GridCopy, N) == 0 and Next:
    #If the value of the entry is 0, we will proceed.

        for NSolved in range(1, 10):
        #The value of the entry, once solved, can only be between 1 and 9. 
            
            #We check every number until we find one fitting the following condition:
            if NSolved not in SubGrid(Grid, N) and NSolved not in Row(Grid, N) and NSolved not in Column(Grid, N):
            #We check if the value of the entry is not already in the subgrid, row and column. 
                #If not, we can proceed by updating the value:
                Grid[IR(N)][IC(N)] = NSolved
                #We itterate, and exit the loop:
                N += 1
                break

            #If the value of the entry already exists in the subgrid, row or column:
            else:
                if NSolved == 9:
                #We reach the end of the allowed range without any valid value.
                    #We need to go back to the previous entry:
                    #We update the variables to go backward, and exit the loop:
                    Next = False
                    Previous = True
                    N -= 1
                    break

#Backwards conditions :

    elif NValue(GridCopy, N) != 0 and Previous:
        N -= 1
        #If the value of the entry is not 0, the value is a pre-established one, and we skip it, going backwards once more.
    
    #If the value of the entry is 0:
    elif NValue(GridCopy, N) == 0 and Previous:
        
        if Grid[IR(N)][IC(N)] == 9:
            Grid[IR(N)][IC(N)] = 0
            #If the value of N is not a pre-established one but already reached 9, we set it back to 0 because we can't go higher. Then, we go backwards once more:
            N -= 1
        
        else:
            for NSolved in range(NValue(Grid, N)+1, 10):
            #The value of N, once solved, can only be between 1 and 9. Since we already went backwards, we are testing a new value incrementing (one by one) the previous value up to 9.
               
                if NSolved not in SubGrid(Grid, N) and NSolved not in Row(Grid, N) and NSolved not in Column(Grid, N):
                #We check if the value of the entry is not already in the subgrid, row and column. 
                    #If not, we can proceed by updating the value:
                    Grid[IR(N)][IC(N)] = NSolved
                    #We update the variables to go forward, and exit the loop:
                    Next = True
                    Previous = False
                    N += 1
                    break

                #If the new value of the entry already exists in the subgrid, row or column:
                else:
                    if NSolved == 9:
                    #We reach the end of the allowed range without any new valid value.
                        #We need to reset the value to 0 and go back the previous entry, then exit the loop:
                        Grid[IR(N)][IC(N)] = 0
                        N -= 1
                        break

#Once the loop is over, we print the results:
print("The sudoku is solved!")
print("Solution:")
print(*Grid, sep="\n")